In [1]:
# Data exploration
import pandas as pd

# Numerical
import numpy as np

In [2]:
#import required Python scripts to access their functions
import NYC_GetCleaned_HistoricData 
import data_utility
import NYC_GetCleaned_TotalPopulation

In [3]:
#import the functions from their corresponding files
from NYC_GetCleaned_HistoricData import getCleanedData
from NYC_GetCleaned_TotalPopulation import getMeanPopulation

In [4]:
crimes_original = getCleanedData()
crimes_original.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,3/30/2014,20:40:00,3/30/2014,20:50:00,3/30/2014,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE,INTENT TO",ATTEMPTED,FELONY,BRONX,44,INSIDE,RESIDENCE - APT. HOUSE,40.8362,-73.9152
1,4/16/2014,22:10:00,4/16/2014,22:12:00,4/17/2014,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE,POSSESS.",ATTEMPTED,FELONY,BRONX,47,INSIDE,STREET,40.8947,-73.8614
2,4/8/2014,13:45:00,4/8/2014,13:50:00,4/8/2014,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE, SALE 5",ATTEMPTED,FELONY,BRONX,52,INSIDE,STREET,40.8642,-73.9012
3,5/17/2014,21:17:00,5/17/2014,21:24:00,5/17/2014,FELONY ASSAULT,"ASSAULT 2,1,PEACE OFFICER",ATTEMPTED,FELONY,BRONX,40,INSIDE,STREET,40.8131,-73.9082
4,4/26/2014,00:00:00,4/26/2014,00:01:00,4/26/2014,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,41,INSIDE,STREET,40.8146,-73.8859


In [5]:
crime_filter = crimes_original[7].value_counts(dropna=False)


In [6]:
from data_utility import filterData,getCount,sortValue
completed_crimes = filterData(crimes_original,7,'COMPLETED')



In [30]:
dropped_columns = completed_crimes.drop([0,1,2,3,4,6,11,13,14,5,7,10],axis=1)
dropped_columns.columns = ['TYPE OF CRIME','BOROUGHS','PREMISES']
dropped_columns.head()

,TYPE OF CRIME,BOROUGHS,PREMISES
2164,FELONY,BRONX,BAR/NIGHT CLUB
2165,FELONY,QUEENS,nan
2166,FELONY,MANHATTAN,OTHER
2167,MISDEMEANOR,QUEENS,RESIDENCE-HOUSE
2168,MISDEMEANOR,MANHATTAN,OTHER


In [31]:
#Since population count is carried once in a decade, get the mean population of the year 2010 and 2020 for each borough
bronxPop = getMeanPopulation('Bronx', 1)
brookylnPop = getMeanPopulation('Brooklyn', 2)
manhattanPop = getMeanPopulation('Manhattan', 3)
queensPop = getMeanPopulation('Queens', 4)
statIslandPop = getMeanPopulation('Staten Island', 5)


In [32]:
#Set Type of crime to a variable
felony_val = 'FELONY'
mis_val = 'MISDEMEANOR'
vio_val = 'VIOLATION'

#Fetch crimes that are categorized under Felony
data = filterData(dropped_columns,'TYPE OF CRIME',felony_val)
data.head()

,TYPE OF CRIME,BOROUGHS,PREMISES
2164,FELONY,BRONX,BAR/NIGHT CLUB
2165,FELONY,QUEENS,nan
2166,FELONY,MANHATTAN,OTHER
2171,FELONY,BRONX,STREET
2176,FELONY,MANHATTAN,STREET


In [38]:
def createPivotTable(data,levelName):
    
    #set level name for pivot table
    data.columns = [levelName,'BOROUGHS','PREMISES']
    
    #Convert the Felony dataset into a pivot table as per Boroughs and Premises
    fel_pivot = data.pivot_table(values = [levelName], index = ['PREMISES'], columns = ['BOROUGHS'], aggfunc=np.size,fill_value=0,margins=True)
    
    #Convert the values from int64 to float64
    for i in range(6):
        fel_pivot[fel_pivot.columns[0][0], fel_pivot.columns[i][1]] = fel_pivot[fel_pivot.columns[0][0],fel_pivot.columns[i][1]].astype('float64')
        
    #Divide the mean with the number of crimes in each premises for the corresponding Borough
    for idx, row in fel_pivot.iterrows():
                  row[0]=round(row[0]/bronxPop,6)
                  row[1]=round(row[1]/brookylnPop,6)
                  row[2]=round(row[2]/manhattanPop,6)
                  row[3]=round(row[3]/queensPop,6)
                  row[4]=round(row[4]/statIslandPop,6)
                    
    #Sort the pivot table
    sort_pivot = fel_pivot.reindex(fel_pivot[levelName].sort_values(by='All', ascending=False).index)
    
    
    return sort_pivot
    

In [39]:
createPivotTable(data,felony_val)

FELONY                                \
BOROUGHS                         BRONX  BROOKLYN MANHATTAN    QUEENS   
PREMISES                                                               
All                           0.229717  0.209611  0.264045  0.159015   
STREET                        0.090694  0.077899  0.070797  0.059057   
RESIDENCE - APT. HOUSE        0.058381  0.047478  0.052996  0.025144   
RESIDENCE-HOUSE               0.012419  0.022086  0.001699  0.032852   
RESIDENCE - PUBLIC HOUSING    0.015434  0.014463  0.016752  0.003754   
COMMERCIAL BUILDING           0.003411  0.004061  0.015890  0.003954   
TRANSIT - NYC SUBWAY          0.005662  0.004815  0.013385  0.002294   
OTHER                         0.008899  0.004723  0.005867  0.003438   
BAR/NIGHT CLUB                0.001160  0.001720  0.011929  0.001848   
RESTAURANT/DINER              0.001379  0.001891  0.010038  0.001492   
CHAIN STORE                   0.001439  0.001831  0.006566  0.001545   
GROCERY/BODEGA                0.002816  0.002764  0.002334  0.001622   
DEPARTMENT STORE              0.001191  0.001433  0.005802  0.001265   
PARK/PLAYGROUND               0.002254  0.001782  0.003579  0.001333   
BANK                          0.001280  0.001336  0.003497  0.001339   
nan                           0.002057  0.001491  0.002664  0.000969   
CLOTHING/BOUTIQUE             0.000595  0.000732  0.005039  0.000551   
PUBLIC SCHOOL                 0.001896  0.001517  0.001465  0.000893   
PARKING LOT/GARAGE (PRIVATE)  0.001522  0.001176  0.000736  0.001727   
PARKING LOT/GARAGE (PUBLIC)   0.001140  0.001105  0.000725  0.001458   
HOTEL/MOTEL                   0.000233  0.000275  0.004628  0.000521   
PUBLIC BUILDING               0.001498  0.001014  0.001955  0.000557   
FAST FOOD                     0.000973  0.000934  0.002100  0.000641   
GYM/FITNESS FACILITY          0.000547  0.000574  0.003110  0.000454   
STORE UNCLASSIFIED            0.000812  0.000950  0.001415  0.000723   
HOSPITAL                      0.000900  0.000776  0.001821  0.000445   
SMALL MERCHANT                0.000643  0.000751  0.001648  0.000606   
DRUG STORE                    0.000479  0.000505  0.001872  0.000491   
CONSTRUCTION SITE             0.000525  0.000656  0.001036  0.000384   
FOOD SUPERMARKET              0.000549  0.000626  0.000865  0.000473   
...                                ...       ...       ...       ...   
JEWELRY                       0.000140  0.000170  0.000764  0.000150   
CANDY STORE                   0.000314  0.000259  0.000341  0.000156   
PRIVATE/PAROCHIAL SCHOOL      0.000243  0.000250  0.000434  0.000131   
BUS STOP                      0.000323  0.000223  0.000125  0.000269   
VARIETY STORE                 0.000256  0.000311  0.000261  0.000161   
TAXI (LIVERY LICENSED)        0.000356  0.000235  0.000269  0.000136   
HIGHWAY/PARKWAY               0.000208  0.000112  0.000084  0.000382   
TAXI (YELLOW LICENSED)        0.000078  0.000109  0.000663  0.000080   
AIRPORT TERMINAL              0.000004  0.000004  0.000007  0.000690   
TRANSIT FACILITY (OTHER)      0.000143  0.000097  0.000408  0.000102   
SHOE                          0.000067  0.000096  0.000552  0.000070   
SOCIAL CLUB/POLICY            0.000136  0.000149  0.000267  0.000110   
BUS TERMINAL                  0.000012  0.000007  0.000669  0.000019   
LIQUOR STORE                  0.000107  0.000154  0.000169  0.000103   
BUS (OTHER)                   0.000100  0.000090  0.000140  0.000073   
SYNAGOGUE                     0.000021  0.000182  0.000066  0.000047   
BRIDGE                        0.000124  0.000047  0.000165  0.000036   
VIDEO STORE                   0.000044  0.000074  0.000165  0.000067   
MARINA/PIER                   0.000037  0.000056  0.000202  0.000040   
TUNNEL                        0.000090  0.000073  0.000086  0.000084   
ABANDONED BUILDING            0.000040  0.000094  0.000050  0.000048   
BOOK/CARD                     0.000008  0.000039  0.000202  0.000027   


In [40]:
#Fetch crimes that are categorized under Misdemeanor
data_mis = filterData(dropped_columns,'TYPE OF CRIME',mis_val)
createPivotTable(data_mis,mis_val)

MISDEMEANOR                                \
BOROUGHS                           BRONX  BROOKLYN MANHATTAN    QUEENS   
PREMISES                                                                 
All                             0.538753  0.367784  0.496241  0.261625   
STREET                          0.205924  0.136192  0.149715  0.092603   
RESIDENCE - APT. HOUSE          0.133045  0.073206  0.071594  0.039149   
RESIDENCE - PUBLIC HOUSING      0.057131  0.032264  0.054335  0.007031   
RESIDENCE-HOUSE                 0.024999  0.031498  0.002437  0.045305   
DEPARTMENT STORE                0.009137  0.007408  0.022713  0.007953   
CHAIN STORE                     0.008097  0.007976  0.023771  0.006381   
OTHER                           0.015318  0.009500  0.012141  0.007434   
COMMERCIAL BUILDING             0.006560  0.007172  0.020526  0.006605   
TRANSIT - NYC SUBWAY            0.005177  0.006378  0.018295  0.002838   
PARK/PLAYGROUND                 0.007443  0.004042  0.008411  0.002478   
GROCERY/BODEGA                  0.006214  0.005680  0.005565  0.002902   
PUBLIC SCHOOL                   0.006731  0.004747  0.004819  0.003084   
DRUG STORE                      0.003783  0.003093  0.010348  0.003022   
CLOTHING/BOUTIQUE               0.001824  0.001920  0.014375  0.001454   
RESTAURANT/DINER                0.002396  0.002686  0.009753  0.002226   
BAR/NIGHT CLUB                  0.001627  0.001902  0.010277  0.002659   
FOOD SUPERMARKET                0.003442  0.002649  0.004294  0.001662   
PUBLIC BUILDING                 0.004730  0.002197  0.003821  0.001181   
PARKING LOT/GARAGE (PUBLIC)     0.002862  0.002104  0.001554  0.002626   
PARKING LOT/GARAGE (PRIVATE)    0.003272  0.001959  0.001492  0.002713   
nan                             0.003540  0.001328  0.003842  0.001164   
STORE UNCLASSIFIED              0.002164  0.001781  0.002538  0.001490   
SMALL MERCHANT                  0.001396  0.001480  0.003879  0.001128   
FAST FOOD                       0.001896  0.001552  0.003251  0.001084   
HOSPITAL                        0.001926  0.001512  0.002659  0.000803   
HOTEL/MOTEL                     0.000452  0.000436  0.004298  0.000770   
HIGHWAY/PARKWAY                 0.001638  0.001249  0.000877  0.000988   
GYM/FITNESS FACILITY            0.000751  0.000674  0.002479  0.000549   
BEAUTY & NAIL SALON             0.001176  0.000963  0.001409  0.000626   
...                                  ...       ...       ...       ...   
CONSTRUCTION SITE               0.000298  0.000456  0.000833  0.000283   
PRIVATE/PAROCHIAL SCHOOL        0.000445  0.000395  0.000635  0.000260   
TAXI (YELLOW LICENSED)          0.000196  0.000271  0.001244  0.000209   
FACTORY/WAREHOUSE               0.000437  0.000516  0.000111  0.000386   
TRANSIT FACILITY (OTHER)        0.000314  0.000220  0.000560  0.000236   
BUS TERMINAL                    0.000013  0.000014  0.001594  0.000064   
TAXI (LIVERY LICENSED)          0.000552  0.000296  0.000399  0.000218   
CHECK CASHING BUSINESS          0.000607  0.000284  0.000356  0.000165   
LIQUOR STORE                    0.000269  0.000338  0.000415  0.000263   
SHOE                            0.000150  0.000206  0.000838  0.000145   
STORAGE FACILITY                0.000537  0.000320  0.000194  0.000208   
BUS STOP                        0.000391  0.000180  0.000191  0.000258   
TUNNEL                          0.000234  0.000179  0.000300  0.000286   
SOCIAL CLUB/POLICY              0.000184  0.000230  0.000370  0.000171   
ABANDONED BUILDING              0.000145  0.000201  0.000263  0.000166   
JEWELRY                         0.000115  0.000132  0.000444  0.000122   
ATM                             0.000126  0.000120  0.000464  0.000097   
BOOK/CARD                       0.000040  0.000069  0.000554  0.000045   
BUS (OTHER)                     0.000160  0.000110  0.000241  0.000113   
VIDEO STORE                     0.000109  0.000117  0.000291  0.000094   
SYNAGOGUE                       0

In [41]:
#Fetch crimes that are categorized under Violation
data_vio = filterData(dropped_columns,'TYPE OF CRIME',vio_val)
createPivotTable(data_vio,vio_val)

VIOLATION                                \
BOROUGHS                        BRONX  BROOKLYN MANHATTAN    QUEENS   
PREMISES                                                              
All                          0.112467  0.085937  0.092424  0.066775   
RESIDENCE - APT. HOUSE       0.047582  0.029309  0.030622  0.018601   
STREET                       0.022076  0.017931  0.019670  0.013337   
RESIDENCE-HOUSE              0.010520  0.013005  0.000828  0.019213   
RESIDENCE - PUBLIC HOUSING   0.011277  0.008769  0.012419  0.002839   
PUBLIC SCHOOL                0.004294  0.002307  0.002929  0.001475   
OTHER                        0.003295  0.002219  0.002729  0.001630   
COMMERCIAL BUILDING          0.001758  0.001726  0.003884  0.001589   
TRANSIT - NYC SUBWAY         0.000994  0.000870  0.002594  0.000441   
GROCERY/BODEGA               0.001222  0.001105  0.000971  0.000518   
RESTAURANT/DINER             0.000532  0.000619  0.001779  0.000565   
HOSPITAL                     0.000799  0.000595  0.001056  0.000412   
PUBLIC BUILDING              0.000886  0.000595  0.000906  0.000351   
CHAIN STORE                  0.000501  0.000516  0.000994  0.000390   
BAR/NIGHT CLUB               0.000219  0.000339  0.001477  0.000435   
PARK/PLAYGROUND              0.000529  0.000526  0.000952  0.000371   
FAST FOOD                    0.000394  0.000388  0.000597  0.000269   
DOCTOR/DENTIST OFFICE        0.000343  0.000316  0.000442  0.000246   
nan                          0.000365  0.000260  0.000561  0.000227   
BUS (NYC TRANSIT)            0.000417  0.000315  0.000300  0.000224   
HOTEL/MOTEL                  0.000082  0.000112  0.001010  0.000233   
BEAUTY & NAIL SALON          0.000359  0.000317  0.000364  0.000233   
STORE UNCLASSIFIED           0.000295  0.000313  0.000307  0.000236   
DEPARTMENT STORE             0.000318  0.000284  0.000374  0.000162   
SMALL MERCHANT               0.000246  0.000227  0.000376  0.000221   
PARKING LOT/GARAGE (PUBLIC)  0.000217  0.000171  0.000181  0.000235   
FOOD SUPERMARKET             0.000245  0.000212  0.000284  0.000156   
CLOTHING/BOUTIQUE            0.000156  0.000177  0.000446  0.000103   
DRUG STORE                   0.000160  0.000176  0.000310  0.000143   
CHURCH                       0.000148  0.000162  0.000270  0.000169   
...                               ...       ...       ...       ...   
VARIETY STORE                0.000064  0.000080  0.000071  0.000047   
CONSTRUCTION SITE            0.000032  0.000053  0.000120  0.000045   
TAXI (YELLOW LICENSED)       0.000015  0.000018  0.000211  0.000022   
SOCIAL CLUB/POLICY           0.000054  0.000054  0.000079  0.000037   
FERRY/FERRY TERMINAL         0.000001  0.000000  0.000048  0.000000   
CHECK CASHING BUSINESS       0.000093  0.000049  0.000047  0.000028   
STORAGE FACILITY             0.000075  0.000055  0.000044  0.000035   
HIGHWAY/PARKWAY              0.000055  0.000028  0.000025  0.000048   
TAXI (LIVERY LICENSED)       0.000040  0.000034  0.000062  0.000030   
JEWELRY                      0.000025  0.000024  0.000094  0.000020   
SYNAGOGUE                    0.000006  0.000077  0.000022  0.000021   
LIQUOR STORE                 0.000037  0.000035  0.000045  0.000027   
TRANSIT FACILITY (OTHER)     0.000024  0.000024  0.000045  0.000017   
SHOE                         0.000021  0.000024  0.000062  0.000015   
MOSQUE                       0.000038  0.000022  0.000012  0.000024   
TUNNEL                       0.000027  0.000022  0.000019  0.000021   
BUS TERMINAL                 0.000004  0.000007  0.000059  0.000007   
OTHER HOUSE OF WORSHIP       0.000014  0.000017  0.000020  0.000024   
VIDEO STORE                  0.000013  0.000015  0.000031  0.000009   
MARINA/PIER                  0.000010  0.000013  0.000032  0.000003   
BRIDGE                       0.000010  0.000015  0.000020  0.000007   
BOOK/CARD                    0.000008  0.000010  0.000025  0.000008   
PHOTO/COPY                   0.000003  0.000008  0.0